In [35]:
import httplib2
from bs4 import BeautifulSoup, SoupStrainer
import requests

import time

from PyPDF2 import PdfFileReader
import io

import numpy as np
import pandas as pd
import re
import array
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [2]:
# click privacy policy button, pass in webdriver and data safety urls with privacy policy button on page
def press_button(driver, data_url):
    driver.implicitly_wait(1)
    driver.get(data_url)
    print(data_url)
    # press privacy policy button by LINK_TEXT from html/web inspection page
    
    try:
        button = driver.find_element(By.LINK_TEXT, "privacy policy")
        button.click()
    
    except NoSuchElementException:
        pass
        current = "N/A"
        print(current)
        return current
    
    
    # close unused tabs, url for privacy policy is child window
#     p = driver.window_handles[0]
#     c = driver.window_handles[1]
#     driver.switch_to.window(c)
    
#     WebDriverWait(driver, 2)
    
#     current = driver.current_url
#     print(current)
    comp = driver.current_url
    if comp != "https://policies.google.com/privacy" and comp != "https://policies.google.com/privacy?hl=en":
        
        p = driver.window_handles[0]
        c = driver.window_handles[1]
        driver.switch_to.window(c)

        WebDriverWait(driver, 4)

        current = driver.current_url
        print(current)
        # close child window and switch to original/parent window
        driver.close()
    
        driver.switch_to.window(p)
    else:
        WebDriverWait(driver, 2)
        current = driver.current_url
        print(current)

    sleep(1)

    # return privacy policy url for df
    return current

In [3]:
# scroll search page to get all returned app url's and id's
def scroll(driver, timeout):
    last_height = driver.execute_script("return document.body.scrollHeight")
    scroll_pause_time = timeout
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
            
        last_height = new_height

In [4]:
# get urls from search
def get_urls(starts_with):
    http = httplib2.Http()
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--incognito")
    driver = webdriver.Chrome()
    # wait for page to load
    driver.implicitly_wait(30)
    # provide url(s) for driver
    driver.get("https://play.google.com/store/search?q=" + starts_with  + "&c=apps")
    # scroll to get all apps
    scroll(driver, 2)

    # read and obtain all urls
    soup_a = BeautifulSoup(driver.page_source, 'lxml')
    
    driver.close()

    # copy all urls
    urls = []
    for link in soup_a.find_all('a'):
        urls.append(link.get('href'))
    return urls

In [5]:
# alphabet = "abcdefghijklmnopqrstuvwxyz"
# categories = ["Daydream", "Beauty", "Auto & Vehicles", "Business", "Comics", "Books & Reference", "Dating", "Education", "Communication", "Food & Drink", "Entertainment", "Health & Fitness", "Events", "Finance", "Libraries & Demo", "House & Home", "Lifestyle", "Medical", "Maps & Navigation", "News & Magazines", "Audio", "Personalization", "Photography", "Parenting", "Shopping", "Productivity", "Tools", "Social", "Sports", "Video Players & Editors", "Travel & Local", "Weather",
#               "Adventure", "Arcade", "Action", "Board", "Casino", "Casual", "Card", "Music", "Puzzle", "Educational", "Role Playing", "Racing", "Sports", "Strategy", "Simulation", "Brain Games", "Trivia", "Word", "Education", "Creativity", "Video", "TV", "Weather", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"]
categories = ['a']
urls = []
# alphabet = 'ab'

# loop through search categories and add to urls array
for char_index in range(len(categories)):
    print(categories[char_index])
    urls += get_urls(categories[char_index])
    print(urls[char_index])
print(len(urls))
for i in range(len(urls)):
    print(urls[i])


a
/store/games
268
/store/games
https://policies.google.com/privacy
https://myaccount.google.com/termsofservice
/store/search?q=a&c=apps
/store/search?q=a&c=movies
/store/search?q=a&c=books
/store/apps/details?id=com.apple.atve.androidtv.appletv
/store/apps/details?id=com.snapchat.android
/store/apps/details?id=com.discord
/store/apps/details?id=com.zhiliaoapp.musically
/store/apps/details?id=com.innersloth.spacemafia
/store/apps/details?id=com.lockwoodpublishing.avakinlife
/store/apps/details?id=com.roblox.client
/store/apps/details?id=com.whatsapp
/store/apps/details?id=com.google.android.apps.photos
/store/apps/details?id=com.facebook.orca
/store/apps/details?id=com.nianticlabs.pokemongo
/store/apps/details?id=com.tinder
/store/apps/details?id=org.telegram.messenger
/store/apps/details?id=exize.tapas.girlAdrift
/store/apps/details?id=com.miHoYo.GenshinImpact
/store/apps/details?id=com.digidust.elokence.akinator.freemium
/store/apps/details?id=com.chickfila.cfaflagship
/store/apps/de

In [6]:
def get_data_safety_urls(app_ids):
    
    base_url = "https://play.google.com/store/apps/datasafety?id="
    data_urls = []
    
    data_urls = [base_url + s for s in app_ids]
    
    return data_urls
            

In [7]:
# remove extra characters from urls array to get only app ids
app_ids = [val for val in urls if val.startswith('/store/apps/details?id=')]

for i in range(len(app_ids)):
    app_ids[i] = app_ids[i][23:]
    print(app_ids[i])
print(len(app_ids))
print("------------------------------------------------------------------------------------------------------------")
data_urls = get_data_safety_urls(app_ids)
print(len(data_urls))
print(data_urls, app_ids)

com.apple.atve.androidtv.appletv
com.snapchat.android
com.discord
com.zhiliaoapp.musically
com.innersloth.spacemafia
com.lockwoodpublishing.avakinlife
com.roblox.client
com.whatsapp
com.google.android.apps.photos
com.facebook.orca
com.nianticlabs.pokemongo
com.tinder
org.telegram.messenger
exize.tapas.girlAdrift
com.miHoYo.GenshinImpact
com.digidust.elokence.akinator.freemium
com.chickfila.cfaflagship
com.supercell.clashroyale
com.instagram.android
com.nerdyoctopus.gamedots
com.bumble.app
com.viber.voip
com.niksoftware.snapseed
com.google.android.apps.docs
us.zoom.videomeetings
com.spotify.music
com.tuesdayquest.myplanet
com.microsoft.teams
com.amazon.atozm
org.thoughtcrime.securesms
com.netflix.mediaclient
com.google.android.apps.tachyon
com.sunglab.tripleafree
com.tocaboca.tocalifeworld
com.beesquare.almostahero
com.acolorstory
com.global.datealive.gp
com.google.android.apps.youtube.kids
com.twitter.android
com.welikegames.BecameDog
com.waybefore.fastlikeafox
com.gameloft.android.ANM

In [8]:
# pd.set_option('max_colwidth', 400)
# df = pd.DataFrame(data_urls, columns=['data safety urls'])
# display(df)

In [9]:
# appid_df = pd.DataFrame(app_ids, columns=['app id'])
# display(appid_df)

In [10]:
# concatenate app id and data safety urls dfs
df = pd.DataFrame(data_urls, columns=['data safety urls'])
appid_df = pd.DataFrame(app_ids, columns=['app id'])
pd.set_option('max_colwidth', 400)
all_df = pd.concat([appid_df, df], axis=1, ignore_index=True)

all_df.columns = ['app id', 'data safety url']
display(all_df)

,app id,data safety url
0,com.apple.atve.androidtv.appletv,https://play.google.com/store/apps/datasafety?id=com.apple.atve.androidtv.appletv
1,com.snapchat.android,https://play.google.com/store/apps/datasafety?id=com.snapchat.android
2,com.discord,https://play.google.com/store/apps/datasafety?id=com.discord
3,com.zhiliaoapp.musically,https://play.google.com/store/apps/datasafety?id=com.zhiliaoapp.musically
4,com.innersloth.spacemafia,https://play.google.com/store/apps/datasafety?id=com.innersloth.spacemafia
...,...,...
245,com.outfit7.talkingtomgoldrun,https://play.google.com/store/apps/datasafety?id=com.outfit7.talkingtomgoldrun
246,com.companyname.RoomAndHalf,https://play.google.com/store/apps/datasafety?id=com.companyname.RoomAndHalf
247,com.fast.free.unblock.secure.vpn,https://play.google.com/store/apps/datasafety?id=com.fast.free.unblock.secure.vpn
248,com.electricpunch.epicgamemaker,https://play.google.com/store/apps/datasafety?id=com.electricpunch.epicgamemaker


In [11]:
darr = all_df['data safety url'].to_numpy()
print(darr)

['https://play.google.com/store/apps/datasafety?id=com.apple.atve.androidtv.appletv'
 'https://play.google.com/store/apps/datasafety?id=com.snapchat.android'
 'https://play.google.com/store/apps/datasafety?id=com.discord'
 'https://play.google.com/store/apps/datasafety?id=com.zhiliaoapp.musically'
 'https://play.google.com/store/apps/datasafety?id=com.innersloth.spacemafia'
 'https://play.google.com/store/apps/datasafety?id=com.lockwoodpublishing.avakinlife'
 'https://play.google.com/store/apps/datasafety?id=com.roblox.client'
 'https://play.google.com/store/apps/datasafety?id=com.whatsapp'
 'https://play.google.com/store/apps/datasafety?id=com.google.android.apps.photos'
 'https://play.google.com/store/apps/datasafety?id=com.facebook.orca'
 'https://play.google.com/store/apps/datasafety?id=com.nianticlabs.pokemongo'
 'https://play.google.com/store/apps/datasafety?id=com.tinder'
 'https://play.google.com/store/apps/datasafety?id=org.telegram.messenger'
 'https://play.google.com/store/a

In [20]:
# df = pd.DataFrame(columns=['app id'])
# df['app id'] = all_df['app id']
df = pd.DataFrame()
# df = df.fillna('')
# df = df[~df.eq('').all(1)]
# df = df.dropna(columns = ['appId'])
display(df)

""


In [13]:
# press privacy policy button on page
driver = webdriver.Chrome()
driver.implicitly_wait(2)
temp = []
for i in darr:
    temp += press_button(driver, i)
#     urls = urls.append({'privacy_policy_url': str(priv_url)}, ignore_index=True)
#     print(i)
# urls = url.concat(urls, df)
df.insert(loc=0, column = 'privacy_policy_url', value=temp)
driver.close()
# display(urls)

https://play.google.com/store/apps/datasafety?id=com.apple.atve.androidtv.appletv


AttributeError: 'NoneType' object has no attribute 'click'

In [18]:
driver = webdriver.Chrome()
driver.implicitly_wait(2)
temp = press_button(driver, "https://play.google.com/store/apps/datasafety?id=com.apple.atve.androidtv.appletv")
df.insert(loc=1, column = 'privacy policy url', value=temp)
driver.close()

https://play.google.com/store/apps/datasafety?id=com.apple.atve.androidtv.appletv
https://www.apple.com/legal/privacy/en-ww/


In [21]:
display(df)

""


In [64]:
link = "https://www.apple.com/legal/privacy/pdfs/apple-privacy-policy-en-ww.pdf"
policy = []

r = requests.get(link)
f = io.BytesIO(r.content)

reader = PdfFileReader(f)
count = reader.numPages

for i in range(count):
    policy += reader.getPage(i).extractText().split('\n')

In [65]:
print(policy)

['Apple Privacy Policy Apple’s Privacy Policy describes how Apple collects, uses, and shares your personal data. Updated October 27, 2021  ', 'In addition to this Privacy Policy, we provide data and privacy information embedded in our products and certain features that ask to use your personal information. This product-specific information is accompanied by our Data & Privacy Icon.  You will be given an opportunity to review this product-specific information before using these features. You also can view this information at any time, either in settings related to those features and/or online at apple.com/legal/privacy/data.  Please take a moment to familiarize yourself with our privacy practices, accessible via the headings below, and contact us if you have questions. Your California Privacy Disclosures Information Regarding Commercial Electronic Messages in CanadaApple Health Study Apps Privacy Policy ', '', 'What Is Personal Data at Apple? At Apple, we believe strongly in fundamental

In [66]:
# begin working sample of reading privacy policy html page with no dropdown menu's

# url = "https://snap.com/en-US/privacy/privacy-policy"
url = "https://www.whatsapp.com/legal/privacy-policy"

res = requests.get(url)
html_page = res.content

soup = BeautifulSoup(html_page, 'html.parser')
text = soup.find_all(text=True)

In [67]:
set([t.parent.name for t in text])

{'[document]',
 'a',
 'b',
 'body',
 'h1',
 'h3',
 'h4',
 'h5',
 'h6',
 'head',
 'html',
 'i',
 'li',
 'option',
 'p',
 'script',
 'span',
 'title'}

In [100]:
output = ''
blacklist = [
    '[document]',
   'noscript',
    'header',
    'html',
    'meta',
    'head', 
    'input',
    'script',
    'link',
    'style'
]

for t in text:
    if t.parent.name not in blacklist:
        output += '{} '.format(t)

In [101]:
print(output)

# end html page read

Privacy Policy Privacy Policy Select your language azərbaycan Afrikaans Bahasa Indonesia Melayu català čeština dansk Deutsch eesti English español français Gaeilge hrvatski italiano Kiswahili latviešu lietuvių magyar Nederlands norsk bokmål o‘zbek Filipino polski Português (Brasil) Português (Portugal) română shqip slovenčina slovenščina suomi svenska Tiếng Việt Türkçe Ελληνικά български қазақ тілі македонски русский српски українська עברית العربية فارسی اردو বাংলা हिन्दी ગુજરાતી ಕನ್ನಡ मराठी ਪੰਜਾਬੀ தமிழ் తెలుగు മലയാളം ไทย 简体中文 繁體中文（台灣） 繁體中文（香港） 日本語 한국어 Download Features Privacy Help Center Get in touch azərbaycan Afrikaans Bahasa Indonesia Melayu català čeština dansk Deutsch eesti English español français Gaeilge hrvatski italiano Kiswahili latviešu lietuvių magyar Nederlands norsk bokmål o‘zbek Filipino polski Português (Brasil) Português (Portugal) română shqip slovenčina slovenščina suomi svenska Tiếng Việt Türkçe Ελληνικά български қазақ тілі македонски русский српски українська עב